In [13]:
# Imports and Setup
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import json
from typing import List, Dict
from tqdm import tqdm

# Reload modules
import importlib
import exhaustive_solver
import edge_matching
import hierarchical_solver
importlib.reload(edge_matching)
importlib.reload(exhaustive_solver)
importlib.reload(hierarchical_solver)

from puzzle_utils import create_puzzle_pieces, assemble_puzzle, PuzzlePiece, get_complementary_edge
from edge_matching import build_compatibility_matrix
from validation import compute_accuracy_metrics, compute_piece_placement_accuracy
from hierarchical_solver import hierarchical_solve

# Paths
PROJECT_ROOT = Path(".")
DATASET_ROOT = PROJECT_ROOT / "Jigsaw Puzzle Dataset" / "Gravity Falls"
OUTPUT_ROOT = PROJECT_ROOT / "processed_images"
ENHANCED_DIR = OUTPUT_ROOT / "enhanced"
MASK_DIR = OUTPUT_ROOT / "masks"
CORRECT_DIR = DATASET_ROOT / "correct"
ASSEMBLED_DIR = OUTPUT_ROOT / "assembled_8x8"
RESULTS_DIR = PROJECT_ROOT / "results"

ASSEMBLED_DIR.mkdir(exist_ok=True, parents=True)
RESULTS_DIR.mkdir(exist_ok=True, parents=True)

print("✅ 8×8 Puzzle Evaluation Setup Complete")

✅ 8×8 Puzzle Evaluation Setup Complete


In [11]:
# Helper: Greedy Assembly with Beam Search
def greedy_assemble_with_beam(pieces: List[PuzzlePiece], compatibility_matrix: np.ndarray, 
                               grid_size: int, beam_width: int = 7, verbose: bool = False) -> np.ndarray:
    """
    Beam search assembly for quadrants.
    """
    n_pieces = len(pieces)
    edge_names = ['top', 'right', 'bottom', 'left']
    
    # Initialize
    initial_arrangement = np.full((grid_size, grid_size), -1, dtype=int)
    initial_arrangement[0, 0] = 0
    beam = [(initial_arrangement.copy(), {0}, 0.0)]
    
    # Fill positions
    for step in range(1, n_pieces):
        new_candidates = []
        
        for arrangement, placed, cumulative_score in beam:
            # Find empty positions with neighbors
            candidate_positions = []
            
            for row in range(grid_size):
                for col in range(grid_size):
                    if arrangement[row, col] != -1:
                        continue
                    
                    neighbors = []
                    if row > 0 and arrangement[row-1, col] != -1:
                        neighbors.append((arrangement[row-1, col], 'top'))
                    if col > 0 and arrangement[row, col-1] != -1:
                        neighbors.append((arrangement[row, col-1], 'left'))
                    if row < grid_size-1 and arrangement[row+1, col] != -1:
                        neighbors.append((arrangement[row+1, col], 'bottom'))
                    if col < grid_size-1 and arrangement[row, col+1] != -1:
                        neighbors.append((arrangement[row, col+1], 'right'))
                    
                    if neighbors:
                        candidate_positions.append((row, col, neighbors))
            
            if not candidate_positions:
                continue
            
            candidate_positions.sort(key=lambda x: len(x[2]), reverse=True)
            row, col, neighbors = candidate_positions[0]
            
            # Score unplaced pieces
            piece_scores = []
            for piece_id in range(n_pieces):
                if piece_id in placed:
                    continue
                
                total_score = 0
                for neighbor_id, neighbor_edge in neighbors:
                    my_edge = get_complementary_edge(neighbor_edge)
                    my_edge_idx = edge_names.index(my_edge)
                    neighbor_edge_idx = edge_names.index(neighbor_edge)
                    score = compatibility_matrix[piece_id, my_edge_idx, neighbor_id, neighbor_edge_idx]
                    total_score += score
                
                avg_score = total_score / len(neighbors)
                piece_scores.append((piece_id, avg_score))
            
            piece_scores.sort(key=lambda x: x[1], reverse=True)
            
            for piece_id, score in piece_scores[:beam_width]:
                new_arrangement = arrangement.copy()
                new_arrangement[row, col] = piece_id
                new_placed = placed.copy()
                new_placed.add(piece_id)
                new_score = cumulative_score + score
                new_candidates.append((new_arrangement, new_placed, new_score))
        
        new_candidates.sort(key=lambda x: x[2], reverse=True)
        beam = new_candidates[:beam_width]
    
    return beam[0][0] if beam else np.full((grid_size, grid_size), -1, dtype=int)

print("✅ Beam search helper defined")

✅ Beam search helper defined


# Overlap-Based Solver Testing

Testing the new overlap-based hierarchical approach that addresses the main limitation of fixed quadrant boundaries.

In [15]:
# Test Overlap Solver on Sample Puzzles
import overlap_solver
importlib.reload(overlap_solver)
from overlap_solver import solve_8x8_with_overlap

def test_overlap_solver(puzzle_id: int, overlap: int = 1, verbose: bool = True) -> Dict:
    """
    Test overlap solver on a single puzzle.
    
    Args:
        puzzle_id: ID of the puzzle
        overlap: Number of pieces to overlap (1 or 2)
        verbose: Print detailed progress
    """
    puzzle_folder = 'puzzle_8x8'
    grid_size = 8
    
    # Build paths
    original_path = DATASET_ROOT / puzzle_folder / f"{puzzle_id}.jpg"
    enhanced_path = ENHANCED_DIR / f"{puzzle_folder}_{puzzle_id}.jpg"
    mask_path = MASK_DIR / f"{puzzle_folder}_{puzzle_id}.jpg"
    ground_truth_path = CORRECT_DIR / f"{puzzle_id}.png"
    
    if not all([p.exists() for p in [original_path, enhanced_path, mask_path, ground_truth_path]]):
        return {'error': 'Missing files'}
    
    try:
        start_time = time.time()
        
        # Load images
        ground_truth = cv2.imread(str(ground_truth_path))
        
        # Create pieces
        pieces = create_puzzle_pieces(
            str(original_path), str(enhanced_path), str(mask_path), grid_size
        )
        
        # Build compatibility matrix
        compatibility_matrix = build_compatibility_matrix(pieces, strip_width=3, grid_size=grid_size)
        
        # Solve using overlap approach
        arrangement, ssim_score = solve_8x8_with_overlap(
            pieces, compatibility_matrix, ground_truth,
            overlap=overlap, beam_width=7, verbose=verbose
        )
        
        # Assemble
        assembled = assemble_puzzle(pieces, arrangement, grid_size)
        elapsed_time = time.time() - start_time
        
        # Compute additional metrics
        image_metrics = compute_accuracy_metrics(assembled, ground_truth)
        piece_accuracy = compute_piece_placement_accuracy(arrangement, pieces)
        
        # Save
        output_path = ASSEMBLED_DIR / f"{puzzle_id}_overlap_{overlap}.jpg"
        cv2.imwrite(str(output_path), assembled)
        
        return {
            'puzzle_id': puzzle_id,
            'overlap': overlap,
            'ssim': ssim_score,
            'mse': image_metrics['mse'],
            'psnr': image_metrics['psnr'],
            'piece_accuracy': piece_accuracy,
            'elapsed_time': elapsed_time,
            'assembled_path': str(output_path)
        }
    except Exception as e:
        import traceback
        return {'error': str(e), 'puzzle_id': puzzle_id, 'traceback': traceback.format_exc()}

print("✅ Overlap solver test function defined")

✅ Overlap solver test function defined


In [16]:
# Test on 10 Sample Puzzles
print("\n=== Testing Overlap Solver on 10 Sample Puzzles ===\n")

sample_ids = list(range(10))  # Puzzles 0-9
overlap_results = []

for puzzle_id in sample_ids:
    print(f"Testing puzzle {puzzle_id}...", end=" ")
    result = test_overlap_solver(puzzle_id, overlap=1, verbose=False)
    
    if 'error' not in result:
        print(f"✅ SSIM: {result['ssim']:.4f} | Time: {result['elapsed_time']:.1f}s")
        overlap_results.append(result)
    else:
        print(f"❌ Error: {result['error'][:50]}")
        overlap_results.append(result)

# Summary
successful = [r for r in overlap_results if 'error' not in r]
if successful:
    ssim_scores = [r['ssim'] for r in successful]
    times = [r['elapsed_time'] for r in successful]
    
    print("\n" + "="*60)
    print("OVERLAP SOLVER - SAMPLE RESULTS (10 puzzles)")
    print("="*60)
    print(f"Successful: {len(successful)}/10")
    print(f"\nSSIM Scores:")
    print(f"  Mean:   {np.mean(ssim_scores):.4f}")
    print(f"  Median: {np.median(ssim_scores):.4f}")
    print(f"  Min:    {np.min(ssim_scores):.4f}")
    print(f"  Max:    {np.max(ssim_scores):.4f}")
    print(f"\nProcessing Time:")
    print(f"  Mean:   {np.mean(times):.1f}s")
    print(f"  Total:  {np.sum(times)/60:.1f} min")
    
    # Compare to baseline (hierarchical without overlap)
    baseline_ssim = 0.218  # From previous results
    improvement = (np.mean(ssim_scores) - baseline_ssim) / baseline_ssim * 100
    print(f"\nBaseline Comparison:")
    print(f"  Hierarchical (no overlap): {baseline_ssim:.3f}")
    print(f"  Overlap solver:            {np.mean(ssim_scores):.3f}")
    print(f"  Improvement:               {improvement:+.1f}%")
else:
    print("\n⚠️ No successful puzzles to analyze")


=== Testing Overlap Solver on 10 Sample Puzzles ===

Testing puzzle 0... ✅ SSIM: 0.1409 | Time: 19.0s
Testing puzzle 1... ✅ SSIM: 0.1409 | Time: 19.0s
Testing puzzle 1... ✅ SSIM: 0.2061 | Time: 18.5s
Testing puzzle 2... ✅ SSIM: 0.2061 | Time: 18.5s
Testing puzzle 2... ✅ SSIM: 0.1878 | Time: 17.7s
Testing puzzle 3... ✅ SSIM: 0.1878 | Time: 17.7s
Testing puzzle 3... ✅ SSIM: 0.0979 | Time: 19.1s
Testing puzzle 4... ✅ SSIM: 0.0979 | Time: 19.1s
Testing puzzle 4... ✅ SSIM: 0.0798 | Time: 18.4s
Testing puzzle 5... ✅ SSIM: 0.0798 | Time: 18.4s
Testing puzzle 5... ✅ SSIM: 0.0560 | Time: 18.7s
Testing puzzle 6... ✅ SSIM: 0.0560 | Time: 18.7s
Testing puzzle 6... ✅ SSIM: 0.0524 | Time: 17.5s
Testing puzzle 7... ✅ SSIM: 0.0524 | Time: 17.5s
Testing puzzle 7... ✅ SSIM: 0.1915 | Time: 17.0s
Testing puzzle 8... ✅ SSIM: 0.1915 | Time: 17.0s
Testing puzzle 8... ✅ SSIM: 0.1873 | Time: 17.4s
Testing puzzle 9... ✅ SSIM: 0.1873 | Time: 17.4s
Testing puzzle 9... ✅ SSIM: 0.2002 | Time: 17.2s

OVERLAP SOLVER

In [14]:
# Debug: Compare overlap solver with hierarchical solver on single puzzle
print("\n=== Debugging Overlap Solver vs Hierarchical ===\n")

test_id = 2  # Use puzzle 2 which had decent SSIM

# Test hierarchical solver
print("Testing hierarchical solver...")
puzzle_folder = 'puzzle_8x8'
grid_size = 8
original_path = DATASET_ROOT / puzzle_folder / f"{test_id}.jpg"
enhanced_path = ENHANCED_DIR / f"{puzzle_folder}_{test_id}.jpg"
mask_path = MASK_DIR / f"{puzzle_folder}_{test_id}.jpg"
ground_truth_path = CORRECT_DIR / f"{test_id}.png"

ground_truth = cv2.imread(str(ground_truth_path))
pieces = create_puzzle_pieces(str(original_path), str(enhanced_path), str(mask_path), grid_size)
compatibility_matrix = build_compatibility_matrix(pieces, strip_width=3, grid_size=grid_size)

# Hierarchical (baseline) - correct parameter order
hierarchical_result, hier_ssim = hierarchical_solve(
    pieces, compatibility_matrix, grid_size, 
    greedy_assemble_with_beam, ground_truth, verbose=False
)
hier_assembled = assemble_puzzle(pieces, hierarchical_result, grid_size)
hier_metrics = compute_accuracy_metrics(hier_assembled, ground_truth)

print(f"Hierarchical SSIM: {hier_metrics['ssim']:.4f}")

# Overlap
overlap_result, overlap_ssim = solve_8x8_with_overlap(
    pieces, compatibility_matrix, ground_truth, 
    overlap=1, verbose=True
)
overlap_assembled = assemble_puzzle(pieces, overlap_result, grid_size)
overlap_metrics = compute_accuracy_metrics(overlap_assembled, ground_truth)

print(f"\nOverlap SSIM: {overlap_metrics['ssim']:.4f}")
print(f"Difference: {(overlap_metrics['ssim'] - hier_metrics['ssim']):.4f}")


=== Debugging Overlap Solver vs Hierarchical ===

Testing hierarchical solver...
Hierarchical SSIM: 0.1797

=== Solving 8×8 with 1-piece overlap ===
Created 4 overlapping 4×4 regions
  Solving region 1/4 at offset (0, 0)
Hierarchical SSIM: 0.1797

=== Solving 8×8 with 1-piece overlap ===
Created 4 overlapping 4×4 regions
  Solving region 1/4 at offset (0, 0)
    Region SSIM: 0.1440
  Solving region 2/4 at offset (0, 3)
    Region SSIM: 0.1440
  Solving region 2/4 at offset (0, 3)
    Region SSIM: 0.3446
  Solving region 3/4 at offset (3, 0)
    Region SSIM: 0.3446
  Solving region 3/4 at offset (3, 0)
    Region SSIM: 0.1977
  Solving region 4/4 at offset (3, 3)
    Region SSIM: 0.1977
  Solving region 4/4 at offset (3, 3)
    Region SSIM: 0.1851

Merging overlapping regions...
  Merged regions: 49/64 positions filled

Resolving conflicts...
  Found 7 duplicate pieces
  Filling 21 empty positions with 22 unused pieces

✅ Final SSIM: 0.2062

Overlap SSIM: 0.2062
Difference: 0.0264
    

# Multi-Level Hierarchy Testing (8→4→2)

Leveraging the excellent 2×2 solver (75.3% SSIM) to build up to 8×8.

In [19]:
# Test Multi-Level Solver
import multilevel_solver
importlib.reload(multilevel_solver)
from multilevel_solver import solve_8x8_multilevel

def test_multilevel_solver(puzzle_id: int, verbose: bool = True) -> Dict:
    """
    Test multi-level hierarchical solver on a single puzzle.
    """
    puzzle_folder = 'puzzle_8x8'
    grid_size = 8
    
    # Build paths
    original_path = DATASET_ROOT / puzzle_folder / f"{puzzle_id}.jpg"
    enhanced_path = ENHANCED_DIR / f"{puzzle_folder}_{puzzle_id}.jpg"
    mask_path = MASK_DIR / f"{puzzle_folder}_{puzzle_id}.jpg"
    ground_truth_path = CORRECT_DIR / f"{puzzle_id}.png"
    
    if not all([p.exists() for p in [original_path, enhanced_path, mask_path, ground_truth_path]]):
        return {'error': 'Missing files'}
    
    try:
        start_time = time.time()
        
        # Load images
        ground_truth = cv2.imread(str(ground_truth_path))
        
        # Create pieces
        pieces = create_puzzle_pieces(
            str(original_path), str(enhanced_path), str(mask_path), grid_size
        )
        
        # Build compatibility matrix
        compatibility_matrix = build_compatibility_matrix(pieces, strip_width=3, grid_size=grid_size)
        
        # Solve using multi-level approach
        arrangement, ssim_score = solve_8x8_multilevel(
            pieces, compatibility_matrix, ground_truth, verbose=verbose
        )
        
        # Assemble
        assembled = assemble_puzzle(pieces, arrangement, grid_size)
        elapsed_time = time.time() - start_time
        
        # Compute additional metrics
        image_metrics = compute_accuracy_metrics(assembled, ground_truth)
        piece_accuracy = compute_piece_placement_accuracy(arrangement, pieces)
        
        # Save
        output_path = ASSEMBLED_DIR / f"{puzzle_id}_multilevel.jpg"
        cv2.imwrite(str(output_path), assembled)
        
        return {
            'puzzle_id': puzzle_id,
            'ssim': ssim_score,
            'mse': image_metrics['mse'],
            'psnr': image_metrics['psnr'],
            'piece_accuracy': piece_accuracy,
            'elapsed_time': elapsed_time,
            'assembled_path': str(output_path)
        }
    except Exception as e:
        import traceback
        return {'error': str(e), 'puzzle_id': puzzle_id, 'traceback': traceback.format_exc()}

print("✅ Multi-level solver test function defined")

✅ Multi-level solver test function defined


In [20]:
# Test Multi-Level on 10 Sample Puzzles
print("\n=== Testing Multi-Level Solver on 10 Sample Puzzles ===\n")

sample_ids = list(range(10))  # Puzzles 0-9
multilevel_results = []

for puzzle_id in sample_ids:
    print(f"Testing puzzle {puzzle_id}...", end=" ")
    result = test_multilevel_solver(puzzle_id, verbose=False)
    
    if 'error' not in result:
        print(f"✅ SSIM: {result['ssim']:.4f} | Time: {result['elapsed_time']:.1f}s")
        multilevel_results.append(result)
    else:
        print(f"❌ Error: {result['error'][:80]}")
        if 'traceback' in result:
            print(f"   {result['traceback'][:200]}")
        multilevel_results.append(result)

# Summary
successful = [r for r in multilevel_results if 'error' not in r]
if successful:
    ssim_scores = [r['ssim'] for r in successful]
    times = [r['elapsed_time'] for r in successful]
    
    print("\n" + "="*60)
    print("MULTI-LEVEL SOLVER - SAMPLE RESULTS (10 puzzles)")
    print("="*60)
    print(f"Successful: {len(successful)}/10")
    print(f"\nSSIM Scores:")
    print(f"  Mean:   {np.mean(ssim_scores):.4f}")
    print(f"  Median: {np.median(ssim_scores):.4f}")
    print(f"  Min:    {np.min(ssim_scores):.4f}")
    print(f"  Max:    {np.max(ssim_scores):.4f}")
    print(f"\nProcessing Time:")
    print(f"  Mean:   {np.mean(times):.1f}s")
    print(f"  Total:  {np.sum(times)/60:.1f} min")
    
    # Compare to baselines
    hier_baseline = 0.218  # Hierarchical without overlap
    overlap_baseline = 0.140  # Overlap solver
    improvement_vs_hier = (np.mean(ssim_scores) - hier_baseline) / hier_baseline * 100
    improvement_vs_overlap = (np.mean(ssim_scores) - overlap_baseline) / overlap_baseline * 100
    
    print(f"\nBaseline Comparisons:")
    print(f"  Hierarchical (baseline):   {hier_baseline:.3f}")
    print(f"  Overlap solver:            {overlap_baseline:.3f}")
    print(f"  Multi-level (8→4→2):       {np.mean(ssim_scores):.3f}")
    print(f"  vs Hierarchical:           {improvement_vs_hier:+.1f}%")
    print(f"  vs Overlap:                {improvement_vs_overlap:+.1f}%")
else:
    print("\n⚠️ No successful puzzles to analyze")


=== Testing Multi-Level Solver on 10 Sample Puzzles ===

Testing puzzle 0... ✅ SSIM: 0.0963 | Time: 16.5s
Testing puzzle 1... ✅ SSIM: 0.1573 | Time: 16.5s
Testing puzzle 2... ✅ SSIM: 0.1794 | Time: 16.0s
Testing puzzle 3... ✅ SSIM: 0.0997 | Time: 16.3s
Testing puzzle 4... ✅ SSIM: 0.0978 | Time: 15.4s
Testing puzzle 5... ✅ SSIM: 0.0615 | Time: 17.1s
Testing puzzle 6... ✅ SSIM: 0.0683 | Time: 17.7s
Testing puzzle 7... ✅ SSIM: 0.1591 | Time: 15.2s
Testing puzzle 8... ✅ SSIM: 0.2066 | Time: 15.2s
Testing puzzle 9... ✅ SSIM: 0.1795 | Time: 14.7s

MULTI-LEVEL SOLVER - SAMPLE RESULTS (10 puzzles)
Successful: 10/10

SSIM Scores:
  Mean:   0.1305
  Median: 0.1285
  Min:    0.0615
  Max:    0.2066

Processing Time:
  Mean:   16.1s
  Total:  2.7 min

Baseline Comparisons:
  Hierarchical (baseline):   0.218
  Overlap solver:            0.140
  Multi-level (8→4→2):       0.131
  vs Hierarchical:           -40.1%
  vs Overlap:                -6.8%


In [21]:
# Debug Multi-Level - Compare Individual Components
print("\n=== Debugging Multi-Level Components ===\n")

test_id = 2
puzzle_folder = 'puzzle_8x8'
grid_size = 8

# Load puzzle
original_path = DATASET_ROOT / puzzle_folder / f"{test_id}.jpg"
enhanced_path = ENHANCED_DIR / f"{puzzle_folder}_{test_id}.jpg"
mask_path = MASK_DIR / f"{puzzle_folder}_{test_id}.jpg"
ground_truth_path = CORRECT_DIR / f"{test_id}.png"

ground_truth = cv2.imread(str(ground_truth_path))
pieces = create_puzzle_pieces(str(original_path), str(enhanced_path), str(mask_path), grid_size)
compatibility_matrix = build_compatibility_matrix(pieces, strip_width=3, grid_size=grid_size)

# Test hierarchical (baseline)
print("1. Hierarchical (baseline):")
hier_arr, hier_score = hierarchical_solve(pieces, compatibility_matrix, grid_size, 
                                         greedy_assemble_with_beam, ground_truth, verbose=False)
print(f"   SSIM: {hier_score:.4f}")

# Test multi-level
print("\n2. Multi-level (8→4→2):")
ml_result = test_multilevel_solver(test_id, verbose=True)
if 'error' not in ml_result:
    print(f"   Final SSIM: {ml_result['ssim']:.4f}")
else:
    print(f"   Error: {ml_result['error']}")

print(f"\nComparison: Multi-level is {(ml_result['ssim']/hier_score - 1)*100:.1f}% vs Hierarchical")


=== Debugging Multi-Level Components ===

1. Hierarchical (baseline):
   SSIM: 0.1797

2. Multi-level (8→4→2):

=== Multi-Level Hierarchical Solve (8×8 → 4×4 → 2×2) ===

Solving 4×4 quadrant 1/4 using 2×2 blocks...
  Solving 2×2 block 1/4...
  Solving 2×2 block 2/4...
  Solving 2×2 block 3/4...
  Solving 2×2 block 4/4...
  4×4 from 2×2 blocks SSIM: 0.1684
  Quadrant 1 SSIM: 0.1684

Solving 4×4 quadrant 2/4 using 2×2 blocks...
  Solving 2×2 block 1/4...
  Solving 2×2 block 2/4...
  Solving 2×2 block 3/4...
  Solving 2×2 block 4/4...
  4×4 from 2×2 blocks SSIM: 0.1079
  Quadrant 2 SSIM: 0.1079

Solving 4×4 quadrant 3/4 using 2×2 blocks...
  Solving 2×2 block 1/4...
  Solving 2×2 block 2/4...
  Solving 2×2 block 3/4...
  Solving 2×2 block 4/4...
  4×4 from 2×2 blocks SSIM: 0.2124
  Quadrant 3 SSIM: 0.2124

Solving 4×4 quadrant 4/4 using 2×2 blocks...
  Solving 2×2 block 1/4...
  Solving 2×2 block 2/4...
  Solving 2×2 block 3/4...
  Solving 2×2 block 4/4...
  4×4 from 2×2 blocks SSIM: 0.2

In [ ]:
# Process Single 8×8 Puzzle
def process_8x8_puzzle(puzzle_id: int, verbose: bool = False) -> Dict:
    """
    Process a single 8×8 puzzle using hierarchical SSIM-validated approach.
    """
    puzzle_folder = 'puzzle_8x8'
    grid_size = 8
    
    # Build paths
    original_path = DATASET_ROOT / puzzle_folder / f"{puzzle_id}.jpg"
    enhanced_path = ENHANCED_DIR / f"{puzzle_folder}_{puzzle_id}.jpg"
    mask_path = MASK_DIR / f"{puzzle_folder}_{puzzle_id}.jpg"
    ground_truth_path = CORRECT_DIR / f"{puzzle_id}.png"
    
    if not all([p.exists() for p in [original_path, enhanced_path, mask_path, ground_truth_path]]):
        return {'error': 'Missing files'}
    
    try:
        start_time = time.time()
        
        # Load images
        ground_truth = cv2.imread(str(ground_truth_path))
        
        # Create pieces
        pieces = create_puzzle_pieces(
            str(original_path), str(enhanced_path), str(mask_path), grid_size
        )
        
        # Build compatibility matrix
        compatibility_matrix = build_compatibility_matrix(pieces, strip_width=3, grid_size=grid_size)
        
        # Solve using hierarchical approach
        arrangement, best_score = hierarchical_solve(
            pieces, compatibility_matrix, grid_size,
            greedy_assemble_with_beam, ground_truth=ground_truth, verbose=verbose
        )
        
        # Assemble
        assembled = assemble_puzzle(pieces, arrangement, grid_size)
        elapsed_time = time.time() - start_time
        
        # Compute metrics
        image_metrics = compute_accuracy_metrics(assembled, ground_truth)
        piece_accuracy = compute_piece_placement_accuracy(arrangement, pieces)
        
        # Save
        output_path = ASSEMBLED_DIR / f"{puzzle_id}_assembled.jpg"
        cv2.imwrite(str(output_path), assembled)
        
        return {
            'puzzle_id': puzzle_id,
            'ssim': image_metrics['ssim'],
            'mse': image_metrics['mse'],
            'psnr': image_metrics['psnr'],
            'piece_accuracy': piece_accuracy,
            'elapsed_time': elapsed_time,
            'best_score': float(best_score),
            'assembled_path': str(output_path)
        }
    except Exception as e:
        import traceback
        return {'error': str(e), 'puzzle_id': puzzle_id, 'traceback': traceback.format_exc()}

print("✅ 8×8 processing function defined")

In [ ]:
# Test Single Puzzle with Verbose Output
print("\n=== Testing 8×8 Solver (Verbose) ===")
test_result = process_8x8_puzzle(0, verbose=True)

if 'error' not in test_result:
    print(f"\n✅ Test successful:")
    print(f"  SSIM: {test_result['ssim']:.4f}")
    print(f"  Time: {test_result['elapsed_time']:.2f}s")
    print(f"  Internal Score: {test_result['best_score']:.4f}")
else:
    print(f"\n❌ Error: {test_result['error']}")

In [ ]:
# Process All 8×8 Puzzles
results_8x8 = []
TOTAL_PUZZLES = 110

print("\n=== Processing 8×8 Puzzles ===")
start_time = time.time()

for puzzle_id in tqdm(range(TOTAL_PUZZLES), desc="8×8 Puzzles"):
    result = process_8x8_puzzle(puzzle_id, verbose=False)
    results_8x8.append(result)

elapsed = time.time() - start_time

# Filter successful results
successful = [r for r in results_8x8 if 'error' not in r]
failed = [r for r in results_8x8 if 'error' in r]

print(f"\n✅ Complete: {len(successful)}/{TOTAL_PUZZLES} successful")
print(f"⏱️ Total time: {elapsed/60:.1f} min ({elapsed/TOTAL_PUZZLES:.2f}s per puzzle)")

if failed:
    print(f"\n⚠️ {len(failed)} puzzles failed:")
    for r in failed[:3]:
        print(f"  - Puzzle {r['puzzle_id']}: {r['error']}")

In [ ]:
# Analyze Results
if successful:
    ssim_scores = [r['ssim'] for r in successful]
    piece_accuracies = [r['piece_accuracy'] for r in successful]
    times = [r['elapsed_time'] for r in successful]
    
    print("\n" + "="*60)
    print("8×8 PUZZLE RESULTS")
    print("="*60)
    print(f"\nSSIM Scores:")
    print(f"  Mean:   {np.mean(ssim_scores):.4f}")
    print(f"  Median: {np.median(ssim_scores):.4f}")
    print(f"  Std:    {np.std(ssim_scores):.4f}")
    print(f"  Min:    {np.min(ssim_scores):.4f}")
    print(f"  Max:    {np.max(ssim_scores):.4f}")
    
    print(f"\nProcessing Time:")
    print(f"  Mean: {np.mean(times):.2f}s")
    print(f"  Total: {np.sum(times)/60:.1f} min")
    
    print(f"\nBaseline Comparison:")
    baseline = 0.107
    improvement = (np.mean(ssim_scores) - baseline) / baseline * 100
    print(f"  Baseline: {baseline:.3f}")
    print(f"  Current:  {np.mean(ssim_scores):.3f}")
    print(f"  Improvement: +{improvement:.1f}% ({np.mean(ssim_scores)/baseline:.2f}×)")
    
    # Save results
    results_file = RESULTS_DIR / "results_8x8.json"
    with open(results_file, 'w') as f:
        json.dump(results_8x8, f, indent=2)
    print(f"\n💾 Results saved to: {results_file}")

In [ ]:
# Visualize Results
if successful:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('8×8 Puzzle Results - SSIM-Validated Hierarchical', fontsize=14, fontweight='bold')
    
    # SSIM distribution
    axes[0, 0].hist(ssim_scores, bins=30, alpha=0.7, edgecolor='black', color='steelblue')
    axes[0, 0].axvline(np.mean(ssim_scores), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(ssim_scores):.3f}')
    axes[0, 0].axvline(0.107, color='orange', linestyle='--', linewidth=2, label='Baseline: 0.107')
    axes[0, 0].set_xlabel('SSIM Score')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('SSIM Distribution')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # Processing time
    axes[0, 1].hist(times, bins=20, alpha=0.7, edgecolor='black', color='orange')
    axes[0, 1].axvline(np.mean(times), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(times):.2f}s')
    axes[0, 1].set_xlabel('Processing Time (s)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Processing Time Distribution')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    # SSIM box plot
    axes[1, 0].boxplot(ssim_scores)
    axes[1, 0].axhline(0.107, color='orange', linestyle='--', linewidth=2, label='Baseline')
    axes[1, 0].set_ylabel('SSIM Score')
    axes[1, 0].set_title('SSIM Box Plot')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    # Best vs Worst
    sorted_results = sorted(successful, key=lambda x: x['ssim'])
    best_5 = [r['ssim'] for r in sorted_results[-5:]]
    worst_5 = [r['ssim'] for r in sorted_results[:5]]
    
    x = np.arange(5)
    width = 0.35
    axes[1, 1].bar(x - width/2, worst_5, width, label='Worst 5', color='crimson', alpha=0.7)
    axes[1, 1].bar(x + width/2, best_5, width, label='Best 5', color='green', alpha=0.7)
    axes[1, 1].set_ylabel('SSIM Score')
    axes[1, 1].set_title('Best vs Worst Puzzles')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'results_8x8_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Visualization saved")

## Future Improvements

### 1. Adaptive Quadrant Boundaries
- Analyze image content (edges, regions)
- Split along natural boundaries
- Avoid cutting through important features

### 2. Overlap Regions
- Solve quadrants with 1-2 piece overlap
- Use overlap to validate and refine seams
- Weighted averaging in overlap zones

### 3. Multi-Level Hierarchy
- 8×8 → four 4×4 → sixteen 2×2
- Bottom-up assembly from 2×2 blocks
- Better error containment at each level

### 4. Graph-Based Optimization
- Model as minimum cost perfect matching problem
- Use Hungarian algorithm or network flow
- Global optimization vs greedy approach

### 5. Deep Learning Features
- Pre-trained CNN features (ResNet, VGG)
- Learn similarity metric for pieces
- End-to-end trainable solver

### 6. Simulated Annealing
- Start with current solution
- Randomly swap pieces with decreasing probability
- Escape local minima

**Target**: Achieve 35-40% mean SSIM (current: 21.8%)

## Position-Aware Compatibility Testing

Testing position-aware compatibility matrix that boosts scores for pieces near original positions.

In [25]:
# Position-Aware Solver for 8×8 Puzzles
importlib.reload(edge_matching)
from edge_matching import build_position_aware_compatibility_matrix

def solve_8x8_position_aware(puzzle_id: int, beam_width: int = 7, position_weight: float = 0.2, verbose: bool = True) -> Dict:
    """
    Solve 8×8 puzzle using position-aware compatibility matrix.
    
    Args:
        puzzle_id: ID of the puzzle
        beam_width: Beam search width
        position_weight: Weight for position bonus (0.2 = 20% boost)
        verbose: Print detailed progress
    
    Returns:
        Dictionary with result, metrics, and timing
    """
    puzzle_folder = 'puzzle_8x8'
    grid_size = 8
    
    # Build paths
    original_path = DATASET_ROOT / puzzle_folder / f"{puzzle_id}.jpg"
    enhanced_path = ENHANCED_DIR / f"{puzzle_folder}_{puzzle_id}.jpg"
    mask_path = MASK_DIR / f"{puzzle_folder}_{puzzle_id}.jpg"
    ground_truth_path = CORRECT_DIR / f"{puzzle_id}.png"
    
    if not original_path.exists() or not enhanced_path.exists():
        return {"success": False, "error": "Files not found"}
    
    if verbose:
        print(f"\\n{'='*60}")
        print(f"Testing Position-Aware Solver - Puzzle {puzzle_id}")
        print(f"{'='*60}")
    
    start = time.time()
    
    # Create pieces
    pieces = create_puzzle_pieces(
        str(original_path), 
        str(enhanced_path), 
        str(mask_path), 
        grid_size
    )
    
    if verbose:
        print(f"✅ Created {len(pieces)} pieces")
    
    # Build initial arrangement (scrambled)
    initial_arrangement = np.arange(grid_size * grid_size).reshape(grid_size, grid_size)
    
    # Build position-aware compatibility matrix
    compatibility_matrix = build_position_aware_compatibility_matrix(
        pieces, 
        initial_arrangement,
        strip_width=3,
        grid_size=grid_size,
        position_weight=position_weight
    )
    
    if verbose:
        print(f"✅ Built position-aware compatibility matrix (weight={position_weight})")
    
    # Solve with beam search
    result = greedy_assemble_with_beam(pieces, compatibility_matrix, grid_size, beam_width=beam_width)
    
    solve_time = time.time() - start
    
    # Assemble and evaluate
    assembled = assemble_puzzle(pieces, result, grid_size)
    
    # Load ground truth
    ground_truth = cv2.imread(str(ground_truth_path))
    if ground_truth is None:
        ground_truth = cv2.imread(str(original_path))
    
    # Compute metrics
    metrics = compute_accuracy_metrics(assembled, ground_truth)
    placement_acc = compute_piece_placement_accuracy(result, pieces)
    metrics['direct_accuracy'] = placement_acc
    metrics['ssim_score'] = metrics['ssim']
    
    if verbose:
        print(f"\\n📊 Results:")
        print(f"  SSIM Score: {metrics['ssim_score']:.4f}")
        print(f"  Direct Accuracy: {metrics['direct_accuracy']:.2%}")
        print(f"  Solve Time: {solve_time:.2f}s")
    
    return {
        "success": True,
        "puzzle_id": puzzle_id,
        "result": result,
        "assembled": assembled,
        "ground_truth": ground_truth,
        "metrics": metrics,
        "solve_time": solve_time,
        "beam_width": beam_width,
        "position_weight": position_weight
    }

print("✅ Position-aware solver function defined")

✅ Position-aware solver function defined


In [26]:
# Test Position-Aware Solver on Sample Puzzles
sample_ids = [0, 2, 5, 8, 10, 15, 20, 25, 30, 35]

print(f"Testing position-aware solver on {len(sample_ids)} sample puzzles...")
print(f"Beam width: 7, Position weight: 0.2 (20% boost)")

position_results = []
baseline_scores = []

for test_id in sample_ids:
    result = solve_8x8_position_aware(test_id, beam_width=7, position_weight=0.2, verbose=False)
    
    if result["success"]:
        position_results.append(result)
        ssim = result["metrics"]["ssim_score"]
        
        # Also get baseline for comparison (no position weighting)
        baseline_result = solve_8x8_position_aware(test_id, beam_width=7, position_weight=0.0, verbose=False)
        baseline_ssim = baseline_result["metrics"]["ssim_score"]
        baseline_scores.append(baseline_ssim)
        
        improvement = ((ssim - baseline_ssim) / baseline_ssim * 100) if baseline_ssim > 0 else 0
        print(f"Puzzle {test_id:2d}: Baseline={baseline_ssim:.4f}, Position-Aware={ssim:.4f} ({improvement:+.1f}%)")

print(f"\\n{'='*60}")
print(f"Summary:")
print(f"  Baseline mean SSIM: {np.mean(baseline_scores):.4f}")
print(f"  Position-aware mean SSIM: {np.mean([r['metrics']['ssim_score'] for r in position_results]):.4f}")
improvement_pct = ((np.mean([r['metrics']['ssim_score'] for r in position_results]) - np.mean(baseline_scores)) / np.mean(baseline_scores) * 100)
print(f"  Overall improvement: {improvement_pct:+.1f}%")
print(f"{'='*60}")

Testing position-aware solver on 10 sample puzzles...
Beam width: 7, Position weight: 0.2 (20% boost)
Puzzle  0: Baseline=0.0797, Position-Aware=0.0797 (+0.0%)
Puzzle  2: Baseline=0.0948, Position-Aware=0.0948 (+0.0%)
Puzzle  5: Baseline=0.0293, Position-Aware=0.0293 (+0.0%)
Puzzle  8: Baseline=0.0510, Position-Aware=0.0510 (+0.0%)
Puzzle 10: Baseline=0.0766, Position-Aware=0.0766 (+0.0%)
Puzzle 15: Baseline=0.1317, Position-Aware=0.1317 (+0.0%)
Puzzle 20: Baseline=0.1026, Position-Aware=0.1026 (+0.0%)
Puzzle 25: Baseline=0.2861, Position-Aware=0.2861 (+0.0%)
Puzzle 30: Baseline=0.4007, Position-Aware=0.4007 (+0.0%)
Puzzle 35: Baseline=0.0850, Position-Aware=0.0850 (+0.0%)
\n============================================================
Summary:
  Baseline mean SSIM: 0.1337
  Position-aware mean SSIM: 0.1337
  Overall improvement: +0.0%


## Stochastic Solver for 8×8 Puzzles

Adapting the successful 4×4 stochastic approach for 8×8 puzzles with quadrant-biased swaps.

In [29]:
# Load and Test 8×8 Stochastic Solver
import stochastic_solver_8x8
importlib.reload(stochastic_solver_8x8)
from stochastic_solver_8x8 import solve_8x8_stochastic

def test_8x8_stochastic(puzzle_id: int, iterations: int = 1000, verbose: bool = True) -> Dict:
    """
    Test stochastic solver on 8×8 puzzle.
    
    Args:
        puzzle_id: ID of the puzzle
        iterations: Number of stochastic iterations
        verbose: Print detailed progress
    
    Returns:
        Dictionary with result, metrics, and timing
    """
    puzzle_folder = 'puzzle_8x8'
    grid_size = 8
    
    # Build paths
    original_path = DATASET_ROOT / puzzle_folder / f"{puzzle_id}.jpg"
    enhanced_path = ENHANCED_DIR / f"{puzzle_folder}_{puzzle_id}.jpg"
    mask_path = MASK_DIR / f"{puzzle_folder}_{puzzle_id}.jpg"
    ground_truth_path = CORRECT_DIR / f"{puzzle_id}.png"
    
    if not original_path.exists() or not enhanced_path.exists():
        return {"success": False, "error": "Files not found"}
    
    if verbose:
        print(f"\\n{'='*60}")
        print(f"Testing 8×8 Stochastic Solver - Puzzle {puzzle_id}")
        print(f"{'='*60}")
    
    start = time.time()
    
    # Create pieces
    pieces = create_puzzle_pieces(
        str(original_path), 
        str(enhanced_path), 
        str(mask_path), 
        grid_size
    )
    
    if verbose:
        print(f"✅ Created {len(pieces)} pieces")
    
    # Load ground truth
    ground_truth = cv2.imread(str(ground_truth_path))
    if ground_truth is None:
        ground_truth = cv2.imread(str(original_path))
    
    # Solve with stochastic approach
    result = solve_8x8_stochastic(
        pieces, 
        ground_truth, 
        grid_size=grid_size,
        beam_width=7,
        iterations=iterations,
        exploration_rate=0.05,
        verbose=verbose
    )
    
    solve_time = time.time() - start
    
    # Assemble and evaluate
    assembled = assemble_puzzle(pieces, result, grid_size)
    metrics = compute_accuracy_metrics(assembled, ground_truth)
    placement_acc = compute_piece_placement_accuracy(result, pieces)
    metrics['direct_accuracy'] = placement_acc
    metrics['ssim_score'] = metrics['ssim']
    
    if verbose:
        print(f"\\n📊 Final Results:")
        print(f"  SSIM Score: {metrics['ssim_score']:.4f}")
        print(f"  Direct Accuracy: {metrics['direct_accuracy']:.2%}")
        print(f"  Total Solve Time: {solve_time:.2f}s")
    
    return {
        "success": True,
        "puzzle_id": puzzle_id,
        "result": result,
        "assembled": assembled,
        "ground_truth": ground_truth,
        "metrics": metrics,
        "solve_time": solve_time,
        "iterations": iterations
    }

print("✅ 8×8 stochastic solver test function defined")

✅ 8×8 stochastic solver test function defined


In [30]:
# Test on Sample 8×8 Puzzles
sample_ids = [0, 2, 5, 8, 10]  # Start with 5 puzzles for speed

print(f"Testing 8×8 stochastic solver on {len(sample_ids)} sample puzzles...")
print(f"Configuration: 1000 iterations, beam_width=7, exploration_rate=0.05")
print("="*60)

stochastic_8x8_results = []
baseline_8x8_scores = []

for test_id in sample_ids:
    # Get baseline (beam search only)
    pieces_baseline = create_puzzle_pieces(
        str(DATASET_ROOT / 'puzzle_8x8' / f"{test_id}.jpg"),
        str(ENHANCED_DIR / f"puzzle_8x8_{test_id}.jpg"),
        str(MASK_DIR / f"puzzle_8x8_{test_id}.jpg"),
        8
    )
    ground_truth_baseline = cv2.imread(str(CORRECT_DIR / f"{test_id}.png"))
    compatibility_matrix = build_compatibility_matrix(pieces_baseline, strip_width=3, grid_size=8, use_enhanced_features=True)
    baseline_arr = greedy_assemble_with_beam(pieces_baseline, compatibility_matrix, 8, beam_width=7)
    baseline_assembled = assemble_puzzle(pieces_baseline, baseline_arr, 8)
    baseline_metrics = compute_accuracy_metrics(baseline_assembled, ground_truth_baseline)
    baseline_ssim = baseline_metrics['ssim']
    baseline_8x8_scores.append(baseline_ssim)
    
    # Test stochastic solver
    result = test_8x8_stochastic(test_id, iterations=1000, verbose=False)
    
    if result["success"]:
        stochastic_8x8_results.append(result)
        ssim = result["metrics"]["ssim_score"]
        
        improvement = ((ssim - baseline_ssim) / baseline_ssim * 100) if baseline_ssim > 0 else 0
        print(f"Puzzle {test_id:2d}: Baseline={baseline_ssim:.4f}, Stochastic={ssim:.4f} ({improvement:+.1f}%)")

print(f"\\n{'='*60}")
print(f"Summary (5 sample puzzles):")
print(f"  Baseline mean SSIM: {np.mean(baseline_8x8_scores):.4f}")
print(f"  Stochastic mean SSIM: {np.mean([r['metrics']['ssim_score'] for r in stochastic_8x8_results]):.4f}")
improvement_pct = ((np.mean([r['metrics']['ssim_score'] for r in stochastic_8x8_results]) - np.mean(baseline_8x8_scores)) / np.mean(baseline_8x8_scores) * 100)
print(f"  Overall improvement: {improvement_pct:+.1f}%")
print(f"{'='*60}")

Testing 8×8 stochastic solver on 5 sample puzzles...
Configuration: 1000 iterations, beam_width=7, exploration_rate=0.05
Puzzle  0: Baseline=0.0797, Stochastic=0.3307 (+315.0%)
Puzzle  2: Baseline=0.0948, Stochastic=0.3271 (+244.9%)
Puzzle  5: Baseline=0.0293, Stochastic=0.0857 (+192.8%)
Puzzle  8: Baseline=0.0510, Stochastic=0.3569 (+600.2%)
Puzzle 10: Baseline=0.0766, Stochastic=0.2915 (+280.4%)
\n============================================================
Summary (5 sample puzzles):
  Baseline mean SSIM: 0.0663
  Stochastic mean SSIM: 0.2784
  Overall improvement: +320.0%


In [ ]:
# Full Batch Test on 8×8 Puzzles (40 puzzles)
import os
puzzle_files = sorted([f for f in os.listdir(DATASET_ROOT / 'puzzle_8x8') if f.endswith('.jpg')])
puzzle_ids_full = [int(f.split('.')[0]) for f in puzzle_files]

print(f"Running full batch test on {len(puzzle_ids_full)} 8×8 puzzles...")
print(f"Configuration: 1000 iterations per puzzle")
print(f"Estimated time: ~{len(puzzle_ids_full) * 15 / 60:.1f} minutes")
print("="*60)

all_stochastic_results = []
all_baseline_scores = []

start_batch = time.time()

for idx, test_id in enumerate(puzzle_ids_full):
    print(f"Processing puzzle {test_id} ({idx+1}/{len(puzzle_ids_full)})...", end=' ')
    
    try:
        # Get baseline (beam search only)
        pieces_baseline = create_puzzle_pieces(
            str(DATASET_ROOT / 'puzzle_8x8' / f"{test_id}.jpg"),
            str(ENHANCED_DIR / f"puzzle_8x8_{test_id}.jpg"),
            str(MASK_DIR / f"puzzle_8x8_{test_id}.jpg"),
            8
        )
        ground_truth_baseline = cv2.imread(str(CORRECT_DIR / f"{test_id}.png"))
        compatibility_matrix = build_compatibility_matrix(pieces_baseline, strip_width=3, grid_size=8, use_enhanced_features=True)
        baseline_arr = greedy_assemble_with_beam(pieces_baseline, compatibility_matrix, 8, beam_width=7)
        baseline_assembled = assemble_puzzle(pieces_baseline, baseline_arr, 8)
        baseline_metrics = compute_accuracy_metrics(baseline_assembled, ground_truth_baseline)
        baseline_ssim = baseline_metrics['ssim']
        all_baseline_scores.append(baseline_ssim)
        
        # Test stochastic solver
        result = test_8x8_stochastic(test_id, iterations=1000, verbose=False)
        
        if result["success"]:
            all_stochastic_results.append(result)
            ssim = result["metrics"]["ssim_score"]
            improvement = ((ssim - baseline_ssim) / baseline_ssim * 100) if baseline_ssim > 0 else 0
            print(f"Baseline={baseline_ssim:.4f}, Stochastic={ssim:.4f} ({improvement:+.1f}%)")
            
            # Save assembled image
            output_path = ASSEMBLED_DIR / f"{test_id}_stochastic.jpg"
            cv2.imwrite(str(output_path), result["assembled"])
        else:
            print(f"FAILED")
    except Exception as e:
        print(f"ERROR: {e}")

batch_time = time.time() - start_batch

print(f"\\n{'='*60}")
print(f"Full Batch Results ({len(all_stochastic_results)} puzzles):")
print(f"  Baseline mean SSIM: {np.mean(all_baseline_scores):.4f}")
print(f"  Stochastic mean SSIM: {np.mean([r['metrics']['ssim_score'] for r in all_stochastic_results]):.4f}")
improvement_pct = ((np.mean([r['metrics']['ssim_score'] for r in all_stochastic_results]) - np.mean(all_baseline_scores)) / np.mean(all_baseline_scores) * 100)
print(f"  Overall improvement: {improvement_pct:+.1f}%")
print(f"  Processing time: {batch_time:.1f}s ({batch_time/60:.1f} min)")
print(f"  Avg time per puzzle: {batch_time/len(all_stochastic_results):.1f}s")
print(f"{'='*60}")

# Save results
results_file = RESULTS_DIR / 'results_8x8_stochastic.json'
with open(results_file, 'w') as f:
    json.dump({
        'puzzles': [
            {
                'puzzle_id': r['puzzle_id'],
                'ssim_score': float(r['metrics']['ssim_score']),
                'direct_accuracy': float(r['metrics']['direct_accuracy']),
                'solve_time': r['solve_time']
            }
            for r in all_stochastic_results
        ],
        'baseline_scores': [float(s) for s in all_baseline_scores],
        'summary': {
            'mean_ssim': float(np.mean([r['metrics']['ssim_score'] for r in all_stochastic_results])),
            'baseline_mean_ssim': float(np.mean(all_baseline_scores)),
            'improvement_pct': float(improvement_pct),
            'total_time': batch_time
        }
    }, f, indent=2)

print(f"\\n✅ Results saved to {results_file}")

Running full batch test on 110 8×8 puzzles...
Configuration: 1000 iterations per puzzle
Estimated time: ~27.5 minutes
Processing puzzle 0 (1/110)... Baseline=0.0797, Stochastic=0.3682 (+362.1%)
Processing puzzle 1 (2/110)... Baseline=0.1269, Stochastic=0.3185 (+151.0%)
Processing puzzle 10 (3/110)... Baseline=0.0766, Stochastic=0.3762 (+390.9%)
Processing puzzle 100 (4/110)... Baseline=0.1004, Stochastic=0.4053 (+303.6%)
Processing puzzle 101 (5/110)... 